#connect to google drive

In [28]:
#@title
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#load package from google drive

In [29]:
#@title
%reload_ext rpy2.ipython

In [30]:
#@title
%%R
lib_loc <- "/content/drive/MyDrive/Colab_Notebooks/r_lib"
lib_loc

[1] "/content/drive/MyDrive/Colab_Notebooks/r_lib"


In [31]:
#@title
%%R
library(biomaRt, lib=lib_loc)
library(DOSE, lib=lib_loc)
library(enrichplot, lib=lib_loc)
library(clusterProfiler, lib=lib_loc)
library(cowplot, lib=lib_loc)

In [32]:
#@title
%%R  
library(BiocGenerics, lib=lib_loc) 
library(S4Vectors, lib=lib_loc) #TRanges
library(IRanges, lib=lib_loc) #AnnotationDbi
library(Biobase, lib=lib_loc) 
library(AnnotationDbi, lib=lib_loc)
library(org.Hs.eg.db, lib=lib_loc)

In [33]:
#@title
%%R
library(ggnewscale, lib=lib_loc)
library(ggupset, lib=lib_loc)
library(ggridges, lib=lib_loc)
library(europepmc, lib=lib_loc)
library(ggplot2)
library(httr)
library(dplyr) 

#選檔案


In [34]:
import ipywidgets as widgets
from rpy2.robjects import r
import rpy2.robjects as robjects

radiobuttonSpecies = widgets.RadioButtons(
    options= ["mouse", "rat", "human"],
    description='物種:',
    disabled=False
)
def on_radiobuttonSpecies_change(change):
    r.assign("species", change.new)
    #print(change.new)
r.assign("species", "mouse")
display(radiobuttonSpecies)
radiobuttonSpecies.observe(on_radiobuttonSpecies_change, names="value")

RadioButtons(description='物種:', options=('mouse', 'rat', 'human'), value='mouse')

In [35]:
#上傳檔案到colab
from google.colab import files


!rm * -f #delete all temporary file 

file=files.upload()

if (file):
  filename = next(iter(file))
  r = robjects.r
  r.assign('filename', filename) #將檔案名稱python -> R
else:
  print("請再次執行此儲存格，並選擇一個檔案")

rm: cannot remove 'drive': Is a directory
rm: cannot remove 'sample_data': Is a directory


Saving DEP_output.txt to DEP_output.txt


#data

In [36]:
%%R
library(stringr)
fileType <- str_extract_all(filename,'\\b(.txt|.csv)\\b') #確認檔案為csv還是txt
print(fileType)
if (fileType == ".txt") {
    d = read.table(file = filename, header=TRUE ,sep="\t" ,fileEncoding ="UTF-8")
} 
else if (fileType == ".csv")  {
    d = read.csv(file = filename, header=TRUE, fileEncoding ="UTF-8")
}
else {
    print("檔案格式錯誤, 請再選擇一次檔案")
}
print(dim(d))
print(colnames(d))
d[,'ID'] <- sub("-.*", "", d$ID)
#write.csv(d[,'ID'], file="sub.csv")
#print(head(d))

ratioCol <- grep( "ratio$" , colnames(d) )
ratioColname <- colnames(d[ratioCol])
numRatio <- length(ratioCol)
print(numRatio)
cat("ratio: ", ratioColname)

[[1]]
[1] ".txt"

[1] 3749   13
 [1] "LA_1"                   "LA_2"                   "LA_3"                  
 [4] "LA_4"                   "ID"                     "LA_vs_DMSO_p.val"      
 [7] "LA_vs_DMSO_p.adj"       "significant"            "LA_vs_DMSO_significant"
[10] "LA_vs_DMSO_ratio"       "DMSO_centered"          "LA_centered"           
[13] "GeneID"                
[1] 1
ratio:  LA_vs_DMSO_ratio

In [37]:
numRatio_py = robjects.r("numRatio")
ratioColname_py = robjects.r("ratioColname")

radiobuttonRatio = widgets.RadioButtons(
    options= ratioColname_py,
    description='Ratio:',
    disabled=False
)
def on_radiobuttonRatio_change(change):
    r.assign("ratioName", change.new)
    
r.assign("ratioName", ratioColname_py[0])
display(radiobuttonRatio)
radiobuttonRatio.observe(on_radiobuttonRatio_change, names="value")

RadioButtons(description='Ratio:', options=('LA_vs_DMSO_ratio',), value='LA_vs_DMSO_ratio')

In [38]:
%%R 

#連接網頁(uniprot -> entrez)
ID_string <- paste(d$ID, collapse=' ')

#送出資料
library(httr)
result_1 <- httr::POST("https://www.uniprot.org/uploadlists/", body = list('uploadQuery'=ID_string , 'from'="ACC,ID",'to'="P_ENTREZGENEID")) #c("C0HK80 C0HKD9")
print(paste("result: ",result_1[1]))

#得到轉換結果
table_url <- paste0(result_1[1], ".tab")
result <- httr::GET(table_url)
print(paste("result table: ", result[1]))

#把得到的結果轉成table

result_table <- content(result)
uniprot_entrez <- read.table(text=result_table, col.names=c('ID_uniprot', 'GeneID_entrez'))
uniprot_entrez <- uniprot_entrez[-c(1),]

#去除重複 (因為uniprot可能會有多個entrez => 排序後選最小的)
uniprot_entrez[, 2] <- as.numeric(uniprot_entrez[, 2])
uniprot_entrez <- uniprot_entrez [ order(uniprot_entrez$ID_uniprot, uniprot_entrez$GeneID_entrez),]
uniprot_entrez <- uniprot_entrez[!duplicated(uniprot_entrez$ID_uniprot),]

cat("row * colmun =",dim(uniprot_entrez), "\n")
print(head(uniprot_entrez))
#write.table(uniprot_entrez, file="uniprot_entrez.csv", quote=FALSE, row.names=F)
write.csv(uniprot_entrez, file="uniprot_entrez.csv")

[1] "result:  https://www.uniprot.org/mapping/M20220210F248CABF64506F29A91F8037F07B67D14024C34"
[1] "result table:  https://www.uniprot.org/mapping/M20220210F248CABF64506F29A91F8037F07B67D14024C34.tab"


R[write to console]: No encoding supplied: defaulting to UTF-8.



row * colmun = 3605 2 
     ID_uniprot GeneID_entrez
849      A2A432         72584
3646     A2A5N8        241764
2478     A2A5R2         99371
975      A2A8U2        230917
1707     A2A8Z1        100273
3393     A2ADY9         68817


biomaRt


In [39]:
%%R

if (species != "human") {
    
    rat = useEnsembl("ensembl", dataset = "rnorvegicus_gene_ensembl")
    mouse = useEnsembl("ensembl", dataset = "mmusculus_gene_ensembl")
    human = useEnsembl("ensembl", dataset = "hsapiens_gene_ensembl")

    if (species == "mouse") { 
        #小鼠(entrezid) => 人類(entrez)
        print("species => mouse")
        human_mouse_rat <- getLDS(mart=mouse, attributes=c("entrezgene_id"), filters="entrezgene_id" , values=uniprot_entrez[,"GeneID_entrez"], attributesL= c("entrezgene_id"), martL = human, uniqueRows=T ) #values=uniprots_ID
    }
    else {  
        #大鼠(entrezid) => 人類(entrez)
        print("species => rat")
        human_mouse_rat <- getLDS(mart=rat, attributes=c("entrezgene_id"), filters="entrezgene_id" , values=uniprot_entrez[,"GeneID_entrez"], attributesL= c("entrezgene_id"), martL = human, uniqueRows=T ) #values=uniprots_ID
    }
    print(dim(human_mouse_rat))
}
else {
  print("species => human")
}

R[write to console]: Ensembl site unresponsive, trying asia mirror



[1] "species => mouse"
[1] 3524    2


In [40]:
%%R
#處理重複 (因為mouse, rat entrez 轉 human entrez 可能為一對多)
if (species != "human") {
  colnames(human_mouse_rat) <- c('GeneID_entrez','human')
  human_mouse_rat <-human_mouse_rat[ order(human_mouse_rat$GeneID_entrez, human_mouse_rat$human),]
  human_mouse_rat <- human_mouse_rat[!duplicated(human_mouse_rat$GeneID_entrez),]

  print(dim(human_mouse_rat))
  print(head(human_mouse_rat))   
}

[1] 3465    2
     GeneID_entrez human
243          11306    22
1454         11308 10006
1348         11363    33
2540         11364    34
2293         11370    37
2849         11409    35


In [41]:
%%R 
#inner join d(origin data), human_mouse_rat(GeneID, human), uniprot_entrez(ID_uniprot, GeneID_entrez) 
library(dplyr) 

#temp:entrez(mmu_or_rat)、entrez(human)、ID(uniprot)
if (species != "human") {
  temp <- inner_join( uniprot_entrez, human_mouse_rat, by="GeneID_entrez")
  colnames(temp) <- c('ID', 'GeneID', 'human_entrez')
  temp[, "ID"] <- as.character(temp[, "ID"])
  head(temp)

  #data
  data <- inner_join(temp, d, by="ID")
}
else {
    #human join d, uniprot_entrez
    data <- inner_join(uniprot_entrez, d, by="ID_uniprot")
}
#print(head(data))

cat("uniprot_entrez:",dim(uniprot_entrez), "\n") #(human, mouse, rat)uniprot、entrez
if (species != "human") {
  cat("human_mouse_rat(entrez老鼠_entrez人類):",dim(human_mouse_rat), "\n") #(mouse, rat)entrez、人類entrez
  cat("temp(uniport(老鼠)_entrez(老鼠)_entrez(人類))", dim(temp), "\n")#老鼠uniprot、老鼠entrez、人類entrez
}
cat("data:",dim(data), "\n")

#write.table(uniprot_entrez, file="data.csv", quote=FALSE, row.name=F)
#write.table(human_mouse, file="data.csv", quote=FALSE, row.name=F)
write.table(data, file="dep_output_result.txt", quote=F, row.name=F, sep = "\t")
write.csv(data, file="dep_output_result.csv")

uniprot_entrez: 3605 2 
human_mouse_rat(entrez老鼠_entrez人類): 3465 2 
temp(uniport(老鼠)_entrez(老鼠)_entrez(人類)) 3467 3 
data: 3480 15 


In [42]:
%%R
#geneList
print(colnames(data))
## feature 1: numeric vector
#geneList = data[,'LA_vs_DMSO_ratio'] #-log(pValue) #要改LA_vs_DMSO_ratio
geneList = data[,ratioName] #-log(pValue)
## feature 2: named vector
names(geneList) = as.character(data[, 'human_entrez']) 

## feature 3: decreasing orde
geneList = sort(geneList, decreasing = TRUE)
head(geneList)


 [1] "ID"                     "GeneID.x"               "human_entrez"          
 [4] "LA_1"                   "LA_2"                   "LA_3"                  
 [7] "LA_4"                   "LA_vs_DMSO_p.val"       "LA_vs_DMSO_p.adj"      
[10] "significant"            "LA_vs_DMSO_significant" "LA_vs_DMSO_ratio"      
[13] "DMSO_centered"          "LA_centered"            "GeneID.y"              
  3437   4938   2534  81030 439996   9636 
 1.180  1.020  0.964  0.817  0.802  0.798 


In [43]:
%%R
de <- names(geneList)[abs(log(geneList)) > 1]
edo <- enrichDGN(de)
edo2 <- gseDO(geneList, pvalueCutoff=1)
edox <- setReadable(edo, 'org.Hs.eg.db', 'ENTREZID')

R[write to console]: preparing geneSet collections...

R[write to console]: GSEA analysis...

R[write to console]: leading edge analysis...

R[write to console]: done...



#圖

1.Bar Plot

In [44]:
%%R
barplot_1 <- function(){
    pic1 <- barplot(edo,x = "Count", color="p.adjust", showCategory=20)+ xlab("Count")
    print(pic1)
    save_plot("pic1.png", pic1, base_height = 10, base_aspect_ratio = 1)
}

2.Dot Plot

In [45]:
%%R
dotplot_2 <- function(){
   pic2_1 <- dotplot(edo, showCategory=30) + ggtitle("dotplot for ORA")
   pic2_2 <- dotplot(edo2, showCategory=30) + ggtitle("dotplot for GSEA")
   print(pic2_1)
   print(pic2_2)
   save_plot("pic2_1.png", pic2_1, base_height = 10, base_aspect_ratio = 1)
   save_plot("pic2_2.png", pic2_2, base_height = 10, base_aspect_ratio = 1)
}


 3.Gene-Concept Network

In [46]:
%%R 
gene_concept_network_3 <- function(){
    pic3_1 <- cnetplot(edox, foldChange=geneList)
    ## categorySize can be scaled by 'pvalue' or 'geneNum'
    pic3_2 <- cnetplot(edox, categorySize="pvalue", foldChange=geneList)
    pic3_3 <- cnetplot(edox, foldChange=geneList, circular = TRUE, colorEdge = TRUE)
    print(pic3_1)
    print(pic3_2)
    print(pic3_3)
    pic3_4 <- cnetplot(edox, node_label="category") 
    pic3_5 <- cnetplot(edox, node_label="gene") 
    pic3_6 <- cnetplot(edox, node_label="all") 
    pic3_7 <- cnetplot(edox, node_label="none") 
    print(pic3_4)
    print(pic3_5)
    print(pic3_6)
    print(pic3_7)
    save_plot("pic3_1.png", pic3_1, base_height = 10, base_aspect_ratio = 1)
    save_plot("pic3_2.png", pic3_2, base_height = 10, base_aspect_ratio = 1)
    save_plot("pic3_3.png", pic3_3, base_height = 10, base_aspect_ratio = 1)
    save_plot("pic3_4.png", pic3_4, base_height = 10, base_aspect_ratio = 1)
    save_plot("pic3_5.png", pic3_5, base_height = 10, base_aspect_ratio = 1)
    save_plot("pic3_6.png", pic3_6, base_height = 10, base_aspect_ratio = 1)
    save_plot("pic3_7.png", pic3_7, base_height = 10, base_aspect_ratio = 1)
}

4.Heatmap-like functional classification

In [47]:
%%R
heatmap_like_functional_classification_4 <- function(){
    pic4_1 <- heatplot(edox)
    pic4_2 <- heatplot(edox, foldChange=geneList)
    cowplot::plot_grid(pic4_1, pic4_2, ncol=1, labels=LETTERS[1:2])
    pic4_1 <- pic4_1 + coord_flip()
    pic4_2 <- pic4_2 + coord_flip()
    print(pic4_1)
    print(pic4_2)
    save_plot("pic4_1.png", pic4_1, base_height = 70, base_aspect_ratio = 0.5,limitsize = FALSE)
    save_plot("pic4_2.png", pic4_2, base_height = 10, base_aspect_ratio = 3,limitsize = FALSE)
}

5 Enrichment Map

In [48]:
%%R
enrichment_map_5 <- function(){
  edo <- pairwise_termsim(edo)
  pic5_1 <- emapplot(edo)
  pic5_2 <- emapplot(edo, cex_category=1.5)
  pic5_3 <- emapplot(edo, layout="kk")
  pic5_4 <- emapplot(edo, cex_category=1.5,layout="kk") 
  enrichment <- cowplot::plot_grid(pic5_1, pic5_2, pic5_3, pic5_4,labels=LETTERS[1:4])
  print(pic5_1)
  print(pic5_2)
  print(pic5_3)
  print(pic5_4) 
  save_plot("pic5_1.png", pic5_1, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic5_2.png", pic5_2, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic5_3.png", pic5_3, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic5_4.png", pic5_4, base_height = 10, base_aspect_ratio = 1) 
}


6 Biological theme comparison

In [49]:
%%R
biological_theme_comparison_6 <- function() {
  library(clusterProfiler, lib=lib_loc)
  xx <- compareCluster(data, fun="enrichKEGG",
            organism="hsa", pvalueCutoff=0.05) #pvalueCutoff正常為0.05
  xx <- pairwise_termsim(xx)                     
  pic6_1 <- emapplot(xx)
  pic6_2 <- emapplot(xx, legend_n=2) 
  pic6_3 <- emapplot(xx, pie="count")
  pic6_4 <- emapplot(xx, pie="count", cex_category=1.5, layout="kk")
  #cowplot::plot_grid(p1, p2, p3, p4, ncol=2, labels=LETTERS[1:4])
  print(pic6_1)
  print(pic6_2)
  print(pic6_3)
  print(pic6_4)
  save_plot("pic6_1.png", pic6_1, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic6_2.png", pic6_2, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic6_3.png", pic6_3, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic6_4.png", pic6_4, base_height = 10, base_aspect_ratio = 1)
}


7 UpSet Plot

In [50]:
%%R
upSet_plot_7 <- function() {
    pic7_1 <- upsetplot(edo)

    kk2 <- gseKEGG(   geneList = geneList,
               organism     = 'hsa',
               minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
    pic7_2 <- upsetplot(kk2, n = 10)
    
    print(pic7_1)
    print(pic7_2)
    save_plot("pic7_1.png", pic7_1, base_height = 10, base_aspect_ratio = 1.5)
    save_plot("pic7_2.png", pic7_2, base_height = 10, base_aspect_ratio = 1.5)

}

8 ridgeline plot for expression distribution of GSEA result (X軸)

In [51]:
%%R
ridgeline_plot_for_expression_distribution_of_GSEA_result_8 <- function(){
    pic8 <- ridgeplot(edo2) + xlab("expression distributions of enriched genes (log2FC)")
    print(pic8)
    save_plot("pic8.pdf", pic8, base_height = 20, base_aspect_ratio = 0.6)
}

9 running score and preranked list of GSEA result

In [52]:
%%R
running_score_and_preranked_list_of_GSEA_result_9 <- function(){
  pic9_1 <- gseaplot(edo2, geneSetID = 1, by = "runningScore", title = edo2$Description[1])
  pic9_2 <- gseaplot(edo2, geneSetID = 1, by = "preranked", title = edo2$Description[1])
  pic9_3 <- gseaplot(edo2, geneSetID = 1, title = edo2$Description[1])
  
  #cowplot::plot_grid(pic9_1, pic9_2, pic9_3, ncol=1, labels=LETTERS[1:3])
  pic9_4 <- gseaplot2(edo2, geneSetID = 1, title = edo2$Description[1])
 
  pic9_5 <- gseaplot2(edo2, geneSetID = 1:3)
  pic9_6 <- gseaplot2(edo2, geneSetID = 1:3, pvalue_table = TRUE,
          color = c("#E495A5", "#86B875", "#7DB0DD"), ES_geom = "dot")
  
  pic9_7 <- gseaplot2(edo2, geneSetID = 1:3, subplots = 1) + xlab("Rank")
  pic9_8 <- gseaplot2(edo2, geneSetID = 1:3, subplots = 1:2) + xlab("Rank")
  #pic9_7_8 <- cowplot::plot_grid(pic9_7, pic9_8, ncol=1, labels=LETTERS[1:2])

  pic9_9 <- gsearank(edo2, 1, title = edo2[1, "Description"])

  pic9_10 <- lapply(1:3, function(i){
    anno <- edo2[i, c("NES", "pvalue", "p.adjust")]
    lab <- paste0(names(anno), "=",  round(anno, 3), collapse="\n")
    gsearank(edo2, i, edo2[i, 2]) + xlab("Rank")+ylab("???")+ 
      annotate("text", 5000, edo2[i, "enrichmentScore"]* .75, label =lab, hjust=0, vjust=0)})

#plot_grid(plotlist=pp, ncol=1)
  print(pic9_1)
  print(pic9_2)
  print(pic9_3)
  print(pic9_4)
  print(pic9_5)
  print(pic9_6)
  print(pic9_7)
  print(pic9_8)
  print(pic9_9)
  print(pic9_10)
  save_plot("pic9_1.png", pic9_1, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic9_2.png", pic9_2, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic9_3.png", pic9_3, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic9_4.png", pic9_4, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic9_5.png", pic9_5, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic9_6.png", pic9_6, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic9_7.png", pic9_7, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic9_8.png", pic9_8, base_height = 10, base_aspect_ratio = 1)
  save_plot("pic9_9.png", pic9_9, base_height = 10, base_aspect_ratio = 1)
  #save_plot("pic9_10.png", pic9_10, base_height = 10, base_aspect_ratio = 1)
  
}

#開始畫圖

In [53]:
import ipywidgets as widgets
from IPython.display import display, HTML
output = widgets.Output()

@output.capture(clear_output=True,wait=True) # based on https://github.com/jupyter-widgets/ipywidgets/issues/1846 and https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html
def draw(b):
   if (filename):
      
      print ("1. barplot")
      %R barplot_1()
      print ("2. dotplot")
      %R dotplot_2()
      print ("3. Gene-Concept Network")
      %R gene_concept_network_3()
      print ("4. Heatmap-like functional classification")
      %R heatmap_like_functional_classification_4()
      print ("5. Enrichment Map")
      %R enrichment_map_5()
      print ("6. Biological theme comparison")
      %R biological_theme_comparison_6()
      print ("7. UpSet Plot")
      %R upSet_plot_7()
      print ("8. ridgeline plot for expression distribution of GSEA result")
      %R ridgeline_plot_for_expression_distribution_of_GSEA_result_8()
      print ("9. running score and preranked list of GSEA result")
      %R running_score_and_preranked_list_of_GSEA_result_9()
     #%R print(barplot(edo,x = "Count", color="p.adjust", showCategory=20)+ xlab("Count"))
     
      print("畫完了!!")
   else:
      print("請選擇檔案")
        

        
run_button = widgets.Button(
      description = '開始畫圖'
)

run_button.on_click(draw)
display(run_button)
output

Button(description='開始畫圖', style=ButtonStyle())

Output()